# Avaliação de Sistemas de RI - Recuperação da Informação
**Aluno: Pedro Guedes Braga**


In [49]:
#coding: utf-8

import pandas as pd
import nltk
import operator
import math
import matplotlib.pyplot as pyplot
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import *
nltk.download('stopwords')
nltk.download('punkt') 
import csv
import time
import numpy
from operator import itemgetter
import urllib, json

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
url_csv = "https://raw.githubusercontent.com/PedroGuedesBraga/lab06_ri/master/results.csv"
df = pd.read_csv(url_csv)
df.dropna(inplace=True)  ##Retira campos NA (ex.: NaN, etc)
lista_de_noticias = df.text.unique() #Retorna uma lista onde cada elemento é o texto de uma noticia.
 # http://www.nltk.org/howto/portuguese_en.html - Tutorial StopWords
 # https://pythonspot.com/tokenizing-words-and-sentences-with-nltk/
palavras_a_ignorar = nltk.corpus.stopwords.words('portuguese')
palavras_a_ignorar.append('o')
palavras_a_ignorar.append('a')
palavras_a_ignorar.append('é')
palavras_a_ignorar.append('r')
  
special_chars = [',', '.', '-', '“', "”", ")", "(", ":", "%", "?", "$", "–", ""]



##Auxiliares
##Recebe um token(ja em lowercase) e aplica mais estratégias de tokenizacao, descritas pelas duas funcoes auxiliares acima.
def apply_tokenize_strategy(token):
  return remove_periods((remove_apostrophes(token)))

##Metodo auxiliar que remove apostrofos dos tokens, retornando um token em seu formato sem apostrofo
def remove_apostrophes(token):
  return token.replace("’", "").replace("'","")
  
##Metodo auxiliar que remove pontos em uma abreviacao
def remove_periods(token):
  return token.replace(".", "")

#Funcao auxiliar, que simplesmente resgata todos os documentos no indice
def getAllDocuments(index):
  list_of_documents = []
  for word in index:
    for tupla in index[word]:
      if(tupla[0] not in list_of_documents):
        list_of_documents.append(tupla[0])
        
  return list_of_documents







##Questao 1
##Funcao para construir o índice:
def build_index(documents):
  index = {}
  n = 0
  for document in documents:
    n = n + 1
    tokens = word_tokenize(document)
    for token in tokens:
      if((apply_tokenize_strategy(token.lower()) not in palavras_a_ignorar) and (apply_tokenize_strategy(token.lower()) not in special_chars) and ("," not in token.lower())):  ## Todos os tokens passam a serem tratados em LOWERCASE
        if(apply_tokenize_strategy(token.lower()) in index):
          index[apply_tokenize_strategy(token.lower())].append((n, tokens.count(token)))
        else:
          index[apply_tokenize_strategy(token.lower())] = [(n, tokens.count(token))]
  refactored_index = aux_refactor_index(index)
  return refactored_index


#funcao auxiliar responsavel por refatorar as listas invertidas do indice para retorna-la no formato pedido. - Remove duplicatas de documentos
def aux_refactor_index(index):
  for word in index:
    for tupla in index[word]:
      index[word] = list(index[word])
  return index
    
  
##Constrói o indice##
indice = build_index(lista_de_noticias)  








In [0]:
indice = build_index(lista_de_noticias)
##Calcula o idf
number_of_documents = len(getAllDocuments(indice))
for word in indice:
  nk = len(indice[word])
  idf = numpy.log(number_of_documents / nk)
  indice[word] = list(indice[word]) #Converte set para lista - Só para melhor visualizacao mesmo
  indice[word].append(idf)


words = [word for word in indice][0:10]
doc_freq_list = [indice[word] for word in indice][0:10]
idfs = [indice[word][-1] for word in indice][0:10]


# Versões do modelo vetorial

## Representação binária;

In [0]:
def binary(query, document):
  result = 0 #Score
  query_words = query.split()
  document_words = document.split()
  
  for word in query_words:
    result += (word in document_words)
  
  return result

## TF (lembre-se que esse modelo já está implementado);

In [0]:
def term_frequency(query, document):
  result = 0
  query_words = query.split()
  document_words = document.split()
  
  for word in query_words:
    result += document_words.count(word)
  
  return result


## TF-IDF

In [0]:
def tf_idf(query, document):
  result = 0
  document_words = document.split()
  query_words = query.split()
  
  for word in query_words:
    cwd = document_words.count(word)
    if word in indice:
      result += cwd * indice[word][-1]
      
  return round(result,2) #p nao ficar mt grande

## BM25* (não usaremos Okapi já que os documentos não tem grande variação de tamanho).

In [0]:
def bm25(query, document, k):
  result = 0
  doc_words = document.split()
  query_words = query.split()
  
  words = [word for word in query_words if word in doc_words]
    
  for word in words:
    cwd = doc_words.count(word)
    dfw = 0
    if word in indice:
      dfw = len(indice[word][:-1])
    result += (((k+1) * cwd) / (cwd + k)) * numpy.log10(((number_of_documents+1) / dfw)) if dfw != 0 else 0
  
  return round(result,2) #p n ficar mt grande no resultado

# Q1 - Escolha um documento dentre aqueles da base do aluno Bernardi e crie uma consulta que você acha que tem boas chances de recuperar este documento. Em seguida, avalie os resultados de tal consulta usando a métrica de avaliação Reciprocal Rank ( 10 pts)

### - Consulta escolhida: 'eurico miranda'
### - Documento escolhido: nº 51 na base de dados do aluno Bernardi. - URL: 
https://brasil.elpais.com/brasil/2019/03/12/deportes/1552406937_786128.html

### Logo, o documento nº 51 é o documento considerado relevante. A posição 'r' na fórmula do Reciproval Rank (1/r) corresponde a posicao do documento 51 no ranking obtido pelo algoritmo de recuperacao

In [0]:
#Consulta escolhida:
consulta = 'eurico miranda'


#Retorna uma lista com os top 10 documentos usando os diferentes modelos de recuperacao da informacao
#Retorno: [top_10_bin, top_10,tf, top_10_idf, top_10_bm25]
def retrieve_docs(query):
  n = 1 #doc number
  
  ##Listas de tuplas para cada algoritmo: [(score, doc_nmbr)]
  binary_docs = [] 
  tf_docs = []
  tf_idf_docs = []
  bm25_docs = []
  
  for doc in lista_de_noticias:
    doc = doc.lower()
    n = n + 1
    binary_docs.append((binary(query, doc),n))
    tf_docs.append((term_frequency(query, doc),n))
    tf_idf_docs.append((tf_idf(query, doc), n))
    bm25_docs.append((bm25(query, doc, 20), n))
  
  binary_docs.sort(reverse = True)
  tf_docs.sort(reverse=True)
  tf_idf_docs.sort(reverse=True)
  bm25_docs.sort(reverse=True)
    
  return [binary_docs[0:10], tf_docs[0:10], tf_idf_docs[0:10], bm25_docs[0:10]]

#Top 10 documentos para a consulta 'eurico miranda' usando os diferentes algoritmos de recup.
top_10_eurico_miranda = retrieve_docs(consulta)

- **Representação binária; 2,5 pontos (Análise usando Reciprocal Rank)**

- **Avaliando os resultados:** Usando a métrica Reciprocal Rank, observamos que o valor obtido é: **1** (1/r, sendo r = 1, pois o documento vem na pos.1 do ranking).  (*Ver tabela abaixo*)

In [59]:
pd.DataFrame({
    'Docs recuperados - Repres. Binaria': top_10_eurico_miranda[0]
})

,Docs recuperados - Repres. Binaria
0,"(2, 51)"
1,"(1, 122)"
2,"(1, 120)"
3,"(0, 250)"
4,"(0, 249)"
5,"(0, 248)"
6,"(0, 247)"
7,"(0, 246)"
8,"(0, 245)"
9,"(0, 244)"


- **TF; 2,5 pontos (Análise usando Reciprocal Rank)**

- **Avaliando os resultados:** Usando a métrica Reciprocal Rank, observamos que o valor obtido é: **1** (1/r, sendo r = 1, pois o documento vem na pos.1 do ranking).  (*Ver tabela abaixo*)

In [60]:
pd.DataFrame({
    'Docs recuperados - Term Frequency': top_10_eurico_miranda[1]
})

,Docs recuperados - Term Frequency
0,"(5, 51)"
1,"(3, 120)"
2,"(1, 122)"
3,"(0, 250)"
4,"(0, 249)"
5,"(0, 248)"
6,"(0, 247)"
7,"(0, 246)"
8,"(0, 245)"
9,"(0, 244)"


- **BM25; 2,5 pontos (Análise usando Reciprocal Rank)**

- **Avaliando os resultados:** Usando a métrica Reciprocal Rank, observamos que o valor obtido é: **1** (1/r, sendo r = 1, pois o documento vem na pos.1 do ranking).  (*Ver tabela abaixo*)

In [61]:
pd.DataFrame({
    'Docs recuperados - BM25': top_10_eurico_miranda[3]
})

,Docs recuperados - BM25
0,"(7.5, 51)"
1,"(4.25, 120)"
2,"(1.55, 122)"
3,"(0, 250)"
4,"(0, 249)"
5,"(0, 248)"
6,"(0, 247)"
7,"(0, 246)"
8,"(0, 245)"
9,"(0, 244)"


- **TF-IDF. 2,5 pontos (Análise usando Reciprocal Rank)**

- **Avaliando os resultados:** Usando a métrica Reciprocal Rank, observamos que o valor obtido é: **1** (1/r, sendo r = 1, pois o documento vem na pos.1 do ranking).  (*Ver tabela abaixo*)

In [62]:
pd.DataFrame({
    'Docs recuperados - TF_IDF': top_10_eurico_miranda[2]
})

,Docs recuperados - TF_IDF
0,"(19.2, 51)"
1,"(10.71, 120)"
2,"(3.57, 122)"
3,"(0.0, 250)"
4,"(0.0, 249)"
5,"(0.0, 248)"
6,"(0.0, 247)"
7,"(0.0, 246)"
8,"(0.0, 245)"
9,"(0.0, 244)"


# Q2: A partir do gabarito fornecido em OBS1, calcule o MAP para cada algoritmo abaixo e aponte qual obteve o melhor resultado. Para os cálculos do MAP, considere que um documento é relevante para uma dada consulta se este documento estiver entre os documentos do gabarito para essa consulta, senão ele deve ser considerado irrelevante. (10 pts)

In [63]:
with urllib.request.urlopen("https://raw.githubusercontent.com/PedroGuedesBraga/lab_07_ri/master/results_final.json") as url:
  gabarito = json.loads(url.read().decode())


#Auxiliar
# Pega a URL do documento dado o numero do documento
def get_doc_url(doc_number):
  return df.loc[doc_number - 2].url

# Auxiliar que recebe uma consulta e um numero de documento e retorna True se esse documento é relevante para essa consulta
# ou retorna False caso o documento não seja relevante para essa consulta.
# (Baseia-se no gabarito para dizer se o doc é relevante ou nao).
def isRelevant(doc_number, query):
  isRelevant = False
  doc_url = get_doc_url(doc_number)
  #Primeiro pega o indice na query para poder acessar os documentos relevantes associados a essa query
  if(query in gabarito['query']):
    indice_docs_rel = gabarito['query'].index(query)
    docs_relev = gabarito['docs'][indice_docs_rel]
    for document in docs_relev:
      if (document['URL'] == doc_url): 
        isRelevant = True
  return isRelevant


#Auxiliar: Recebe uma query e um top 10 de documentos recuperados e calcula a Average Precision - AP
def calculate_average_precision_aux(query, top10_docs):
  k = 0
  positive_docs_number = 0
  true_posit_pred_posit_list = []
  for doc in top10_docs:
    k = k + 1
    if(isRelevant(doc[1], query)):
      positive_docs_number = positive_docs_number + 1
      true_posit_pred_posit_list.append(positive_docs_number/k)
  if(positive_docs_number == 0):
    return 0
  else: 
    return (1/positive_docs_number) * (sum(true_posit_pred_posit_list))


# Auxiliar: Recebe uma query e um top 10 de documentos recuperados e calcula a Average Precision - AP 
# Só que nesse, ele calcula a Average Precision - AP para cada algoritmo
def calculate_average_precision(query):
  top_10_docs_each_algorithm = retrieve_docs(query) #Retorna os top 10 documentos para uma consulta em cada algoritmo
  
  top_10_bin = top_10_docs_each_algorithm[0]
  top_10_tf = top_10_docs_each_algorithm[1]
  top_10_bm25 = top_10_docs_each_algorithm[3]
  top_10_tf_idf = top_10_docs_each_algorithm[2]
  return (calculate_average_precision_aux(query, top_10_bin), calculate_average_precision_aux(query, top_10_tf), calculate_average_precision_aux(query, top_10_bm25), calculate_average_precision_aux(query, top_10_tf_idf))

# Calculando o mAP 
def calculate_mean_average_precision(queries):
  averages_precisions = [] #Lista com as AP's para cada algortimo com base em cada query - [(bin, tf, bm25, tf_idf)]
  means = [] #Lista com 4 valores correspondentes aos mAPs de cada algortimo
  for query in queries:
    averages_precisions.append(calculate_average_precision(query))
  sum_aps_bin = 0
  sum_aps_tf = 0
  sum_aps_bm25 = 0
  sum_aps_tf_idf = 0
  
  for aps in averages_precisions:
    sum_aps_bin = sum_aps_bin + aps[0]
    sum_aps_tf = sum_aps_tf + aps[1]
    sum_aps_bm25 = sum_aps_bm25 + aps[2]
    sum_aps_tf_idf = sum_aps_tf_idf + aps[3]
  
  return [[sum_aps_bin / 10], [sum_aps_tf / 10], [sum_aps_bm25 / 10], [sum_aps_tf_idf / 10]]

maps = calculate_mean_average_precision(gabarito['query'])

#Exibindo os resultados dos mAPs para cada algoritmo
pd.DataFrame({
    'MAP - Binary': maps[0],
    '/ MAP - TF': maps[1],
    '/ MAP - BM25': maps[2],
    '/ MAP - TF-IDF': maps[3]
})



    
    
   
  
  
    
  
  

,MAP - Binary,/ MAP - TF,/ MAP - BM25,/ MAP - TF-IDF
0,0.291071,0.074286,0.340873,0.331647


### Observando a tabela anterior, o algoritmo que teve melhor resultado para o MAP foi o BM25